In [1]:
# import necessary modules
import pandas as pd
import requests
import zipfile
import os
from datetime import datetime

In [2]:
# print start of the job
start_time = datetime.now()
print('Data Extraction job started at at {}'.format(start_time))

Data Extraction job started at at 2023-05-24 19:59:40.186644


In [3]:
# Set display option to show all columns
pd.set_option('display.max_columns', None)

In [4]:
# create a folder to hold data from the zipped file
data_folder = 'Data'
# Check if the folder exists
if not os.path.exists(data_folder):
    # Create the folder
    os.makedirs(data_folder)
    print(f'{data_folder} folder created successfully')
else:
    print(f'{data_folder} folder already exists')

Data folder already exists


In [5]:
# create a folder to hold the imported data
reports_folder = 'Reports'
# Check if the folder exists
if not os.path.exists(reports_folder):
    # Create the folder
    os.makedirs(reports_folder)
    print(f'{reports_folder} folder created successfully')
else:
    print(f'{reports_folder} folder already exists')

Reports folder already exists


In [6]:
# url of the API
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"

# retrieve the metadata for pcard-expenditures package and its resources
url = base_url + "/api/3/action/package_show"
params = { "id": "pcard-expenditures"}
package = requests.get(url, params = params, verify=False).json()

C:\Users\mmangoli\AppData\Local\Programs\Python\Python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ckan0.cf.opendata.inter.prod-toronto.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [7]:
package

{'help': 'https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/help_show?name=package_show',
 'success': True,
 'result': {'author': 'pcard@toronto.ca',
  'author_email': 'pcard@toronto.ca',
  'civic_issues': 'Fiscal responsibility',
  'creator_user_id': '329e1506-b545-4fc7-a4ea-e614f220eea7',
  'dataset_category': 'Document',
  'date_published': '2019-07-23 17:52:47.150105',
  'excerpt': 'The dataset contains details of all purchases made by City staff members using City-issued credit cards also referred to as PCards or purchasing cards.   ',
  'formats': 'ZIP,XLS',
  'id': 'ebc3f9c2-2f80-4405-bf4f-5fb309581485',
  'information_url': 'http://www.toronto.ca/finance/index.htm',
  'is_retired': 'false',
  'isopen': False,
  'last_refreshed': '2019-09-03 19:44:00',
  'license_id': 'open-government-licence-toronto',
  'license_title': 'open-government-licence-toronto',
  'maintainer': None,
  'maintainer_email': 'pcard@toronto.ca',
  'metadata_created': '2022-03-10T19:34:25.429521'

In [8]:
package.keys()

dict_keys(['help', 'success', 'result'])

In [9]:
# get the urls with data
data_url = ''
for r in package['result']['resources']:
    print(r['format'],r['url'])
    if r['format'] == 'ZIP':
        data_url = r['url']

print(f'Link for data {data_url}')

XLS https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/ebc3f9c2-2f80-4405-bf4f-5fb309581485/resource/070bdbd3-9bae-4269-b096-e3a8bd7460c8/download/pcard_expenditures_readme.xls
ZIP https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/ebc3f9c2-2f80-4405-bf4f-5fb309581485/resource/d83a5249-fb07-4c38-9145-9e12a32ce1d4/download/expenditures.zip
Link for data https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/ebc3f9c2-2f80-4405-bf4f-5fb309581485/resource/d83a5249-fb07-4c38-9145-9e12a32ce1d4/download/expenditures.zip


In [10]:
# Send a GET request to the URL
data_response = requests.get(data_url, verify=False)

C:\Users\mmangoli\AppData\Local\Programs\Python\Python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ckan0.cf.opendata.inter.prod-toronto.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [11]:
# Save the content of the response to a file
downloaded_zip = 'expenditures.zip'
with open(downloaded_zip, "wb") as file:
    file.write(data_response.content)

In [12]:
# we want to extract the contents of the downloaded zip folder
# Open the ZIP file
with zipfile.ZipFile(downloaded_zip,'r') as zip_obj:
    # extract the data into the data folder we created
    zip_obj.extractall(data_folder)

In [13]:
# create a function to clean column names
def clean_column_names(columns):    
    # replace / with nothing
    cleaned_columns = columns.str.replace('/','')
    # remove double spaces
    cleaned_columns = cleaned_columns.str.replace(' ','')
    # remove full stops
    cleaned_columns = cleaned_columns.str.replace('.','',regex=False)
    # remove hyphens
    cleaned_columns = cleaned_columns.str.replace('-','')
    # remove new lines
    cleaned_columns = cleaned_columns.str.replace('\n','')
    # remove #
    cleaned_columns = cleaned_columns.str.replace('#','')
    # make them upper case
    cleaned_columns = cleaned_columns.str.upper()
    # remove Order
    cleaned_columns = cleaned_columns.str.replace('ORDER','')
    # remove No
    cleaned_columns = cleaned_columns.str.replace('NO','')
    # remove Number
    cleaned_columns = cleaned_columns.str.replace('NUMBER','')
    # remove Work
    cleaned_columns = cleaned_columns.str.replace('WORK','')
    # remove Work
    cleaned_columns = cleaned_columns.str.replace('(MCC)','',regex=False)
    # standardize (WBLS and WBS)
    cleaned_columns = cleaned_columns.str.replace('ORIGINALCURRENCY1','TRANSACTIONCURRENCY') 
    # remove 1
    cleaned_columns = cleaned_columns.str.replace('1','')
    
    # standardize the column names (especially centre and center)
    cleaned_columns = cleaned_columns.str.replace('CENTRE','CENTER')
    # standardize (GL Account and Expense Type Descriptions)
    cleaned_columns = cleaned_columns.str.replace('EXPTYPEDESC','GLACCOUNTDESCRIPTION')
    # standardize (Elelment and Element)
    cleaned_columns = cleaned_columns.str.replace('ELELMENT','ELEMENT')
    # standardize (Descrption and Description)
    cleaned_columns = cleaned_columns.str.replace('DESCRPTION','DESCRIPTION')
    # standardize (Discription and Description)
    cleaned_columns = cleaned_columns.str.replace('DISCRIPTION','DESCRIPTION')
    # standardize (Discriprion and Description)
    cleaned_columns = cleaned_columns.str.replace('DECRIPTION','DESCRIPTION')
    # standardize (Discriprion and Description)
    cleaned_columns = cleaned_columns.str.replace('DISCRIPRION','DESCRIPTION')
    # standardize (WBLS and WBS)
    cleaned_columns = cleaned_columns.str.replace('WBLS','WBS')    
    # standardize (Dt and Date)
    cleaned_columns = cleaned_columns.str.replace('DT','DATE')
    # standardize (Divison and Division)
    cleaned_columns = cleaned_columns.str.replace('DIVISON','DIVISION')
    # standardize (Divison and Division)
    cleaned_columns = cleaned_columns.str.replace('EXPENSE','ACCOUNT')
    # standardize (Amt and Amount)
    cleaned_columns = cleaned_columns.str.replace('AMT','AMOUNT')
    # standardize (Trx and Amount)
    cleaned_columns = cleaned_columns.str.replace('TRX','TRANSACTION')
    # standardize (TrCurrency and Amount)
    cleaned_columns = cleaned_columns.str.replace('TRCURRENCY','TRANSACTIONCURRENCY')
    # standardize (LongText and GLAccountDescription)
    cleaned_columns = cleaned_columns.str.replace('LONGTEXT','GLACCOUNTDESCRIPTION')
    
    # strip spaces from column names
    cleaned_columns = cleaned_columns.str.strip()
    
    
    return cleaned_columns

In [14]:
# import data from the extracted files above

# an empty list that will hold our data
li_df = []
# list of files we extracted
li_files = os.listdir(data_folder)
print(f'we shall import data from {len(li_files)} files')

# initialize a counter
c = 0

for file in li_files:
    # import the data from the files
    df = pd.read_excel(data_folder + '\\' + file)
    
    # clean column names     
    df.columns = clean_column_names(df.columns)
    
    # drop null records
    df.dropna(how='any',inplace=True)
    
    # drop null columns
    df.dropna(how='all',axis=1,inplace=True)
    
    # reset the index
    df.reset_index(drop=True,inplace=True)
    
    # add a column to idenfity which file we extracted the data from
    df['SOURCE_FILE_NAME'] = file
    
    # add the data to our list of dataframes
    li_df.append(df)
    print(f'{c+1}. data from {file} imported successfully. Rows: {df.shape[0]}. Columns: {df.shape[1]}')    
    c += 1

we shall import data from 103 files
1. data from PCard Expenses_201706.xlsx imported successfully. Rows: 5259. Columns: 17
2. data from PCard Expenses_201707.xlsx imported successfully. Rows: 5617. Columns: 17
3. data from PCard Expenses_201708.xlsx imported successfully. Rows: 4761. Columns: 17
4. data from PCard Expenses_201709.xlsx imported successfully. Rows: 3895. Columns: 17
5. data from PCard Expenses_201710.xlsx imported successfully. Rows: 4790. Columns: 17
6. data from PCard Expenses_2017_11_Final.xlsx imported successfully. Rows: 4799. Columns: 17
7. data from PCard Expenses_2017_12_Final.xlsx imported successfully. Rows: 3491. Columns: 17
8. data from PCard Expenses_2018_01_Final_revised.xlsx imported successfully. Rows: 4211. Columns: 17
9. data from PCard Expenses_2018_02_Final.xlsx imported successfully. Rows: 3977. Columns: 17
10. data from PCard Expenses_2018_03_Final.xlsx imported successfully. Rows: 4611. Columns: 17
11. data from PCard Expenses_2018_04_Final.xlsx im

90. data from PCardExpenses_201604.xlsx imported successfully. Rows: 4315. Columns: 17
91. data from PCardExpenses_201605.xlsx imported successfully. Rows: 4358. Columns: 17
92. data from PCardExpenses_201606.xlsx imported successfully. Rows: 4509. Columns: 17
93. data from PCardExpenses_201607.xlsx imported successfully. Rows: 4970. Columns: 17
94. data from PCardExpenses_201608.xlsx imported successfully. Rows: 4680. Columns: 17
95. data from PCardExpenses_201609.xlsx imported successfully. Rows: 3565. Columns: 17
96. data from PCardExpenses_201610.xlsx imported successfully. Rows: 4431. Columns: 17
97. data from PCardExpenses_201611.xlsx imported successfully. Rows: 4094. Columns: 17
98. data from PCardExpenses_201612.xlsx imported successfully. Rows: 3541. Columns: 17
99. data from PCardExpenses_201701.xlsx imported successfully. Rows: 3735. Columns: 17
100. data from PCardExpenses_201702.xlsx imported successfully. Rows: 3854. Columns: 17
101. data from PCardExpenses_201703.xlsx i

In [16]:
# combine the imported data frames into one
combined_df = pd.concat(li_df,ignore_index=True)
print(combined_df.shape)
combined_df

(395616, 17)


,DIVISION,BATCHTRANSACTIONID,TRANSACTIONDATE,CARDPOSTINGDATE,MERCHANTNAME,TRANSACTIONAMOUNT,TRANSACTIONCURRENCY,ORIGINALAMOUNT,ORIGINALCURRENCY,GLACCOUNT,GLACCOUNTDESCRIPTION,COSTCENTERWBSELEMENT,COSTCENTERWBSELEMENTDESCRIPTION,MERCHANTTYPE,MERCHANTTYPEDESCRIPTION,PURPOSE,SOURCE_FILE_NAME
0,PUBLIC HEALTH,4608-1,2017-06-15,2017-06-16,PAYPAL *OBC2012,50.00,CAD,50.00,CAD,4760,MEMBERSHIP FEES,PH3071,MATERNAL INFANT HEALTH PROGRAM SUPPORT,8641.0,"Associations - Civic, Social, and Frater",MEMBERSHIP FEE,PCard Expenses_201706.xlsx
1,PUBLIC HEALTH,4617-1,2017-06-21,2017-06-23,CHNC,423.75,CAD,423.75,CAD,4256,CONFERENCES/SEMINARS - REGISTRATION FEES,PH3071,MATERNAL INFANT HEALTH PROGRAM SUPPORT,8641.0,"Associations - Civic, Social, and Frater",CHNC CONFERENCE,PCard Expenses_201706.xlsx
2,PUBLIC HEALTH,4621-1,2017-06-26,2017-06-27,POST MD-CPD-UOFT,2601.00,CAD,2601.00,CAD,4256,CONFERENCES/SEMINARS - REGISTRATION FEES,PH3071,MATERNAL INFANT HEALTH PROGRAM SUPPORT,8220.0,"Colleges, Universities, Professional Sch",UOFT CONFERENCE,PCard Expenses_201706.xlsx
3,PUBLIC HEALTH,4626-1,2017-06-28,2017-06-29,EVENTBRITE/PERINATALMO,480.66,CAD,480.66,CAD,4256,CONFERENCES/SEMINARS - REGISTRATION FEES,PH3071,MATERNAL INFANT HEALTH PROGRAM SUPPORT,7922.0,Theatrical Producers (except Motion Pict,REGISTRATION FEES,PCard Expenses_201706.xlsx
4,ECONOMIC DEVELOPMENT & CULTURE,4590-1,2017-06-02,2017-06-05,LEE VALLEY - DOWNTOWN,38.31,CAD,38.31,CAD,2600,RECREATIONAL & EDUCATIONAL SUPPLIES,AH0073,HS-MH-EDUCATION/OUTRCH-FORT YORK,5072.0,Hardware Equipment and Supplies,LEMON RASPS,PCard Expenses_201706.xlsx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395611,FIRE SERVICES,4540-3,2017-05-01,2017-05-03,ESSO,55.80,CAD,55.80,CAD,2260,GASOLINE,FR0025,MECHANICAL MAINT-VEHICLE MAINT,5542.0,"Fuel Dispenser, Automated",GASOLINE - SELF SERVICE,PCardExpenses_201705.xlsx
395612,FIRE SERVICES,4545-3,2017-05-03,2017-05-05,ESSO,53.00,CAD,53.00,CAD,2260,GASOLINE,FR0025,MECHANICAL MAINT-VEHICLE MAINT,5542.0,"Fuel Dispenser, Automated",GASOLINE - SELF SERVICE,PCardExpenses_201705.xlsx
395613,FIRE SERVICES,4563-3,2017-05-12,2017-05-16,ESSO,62.50,CAD,62.50,CAD,2260,GASOLINE,FR0025,MECHANICAL MAINT-VEHICLE MAINT,5542.0,"Fuel Dispenser, Automated",GASOLINE - SELF SERVICE,PCardExpenses_201705.xlsx
395614,FIRE SERVICES,4572-3,2017-05-22,2017-05-24,ESSO,54.50,CAD,54.50,CAD,2260,GASOLINE,FR0025,MECHANICAL MAINT-VEHICLE MAINT,5542.0,"Fuel Dispenser, Automated",GASOLINE - SELF SERVICE,PCardExpenses_201705.xlsx


In [17]:
# export the data to a csv file
combined_df.to_csv(f'{reports_folder}\\Combined_pcard_expenditures.csv',index=False)

In [18]:
end_time = datetime.now()
print(f"""
Data Extraction job started at {start_time}.
Data Extraction job finished execution at {end_time}.
Total time taken is for Data Extraction is {end_time-start_time}
""")


Data Extraction job started at 2023-05-24 19:59:40.186644.
Data Extraction job finished execution at 2023-05-24 20:06:32.118265.
Total time taken is for Data Extraction is 0:06:51.931621

